In [4]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output
from scipy.ndimage import gaussian_filter
from dataclasses import dataclass

@dataclass
class VSTConfig:
    """Конфігурація параметрів (State)."""
    a: float
    b: float
    epsilon: float = 1e-10

class VarianceStabilizer:
    """Виконує пряме та зворотне перетворення."""
    def __init__(self, config: VSTConfig):
        self.cfg = config

    def forward(self, image: np.ndarray) -> np.ndarray:
        # Захист від log(0)
        img_safe = np.maximum(image, self.cfg.epsilon)
        # a * log_b(I) = a * ln(I) / ln(b)
        return self.cfg.a * (np.log(img_safe) / np.log(self.cfg.b))

    def inverse(self, transformed_image: np.ndarray) -> np.ndarray:
        # b ^ (I_h / a)
        exponent = transformed_image / self.cfg.a
        return np.power(self.cfg.b, exponent)

    def calc_additive_std(self, speckle_std: float) -> float:
        # sigma_add = (a * sigma_mult) / ln(b)
        return (self.cfg.a * speckle_std) / np.log(self.cfg.b)

class ImageGenerator:
    """Генерація тестових даних Sentinel-1."""
    @staticmethod
    def generate(shape=(400, 400), noise_level=0.25):
        x, y = np.meshgrid(np.linspace(0, 1, shape[0]), np.linspace(0, 1, shape[1]))
        
        # Сигнал: Градієнт + коло + "ріка"
        signal = 100 * (x + y) + 50
        # Коло
        mask_circle = (x - 0.6)**2 + (y - 0.6)**2 < 0.05
        signal[mask_circle] += 150
        # Темна смуга (ріка)
        mask_river = np.abs(x - y - 0.2) < 0.05
        signal[mask_river] *= 0.2

        # чорна пляма:
        mask_black = (x - 0.3)**2 + (y - 0.3)**2 < 0.02
        signal[mask_black] = 1.0
        
        # Мультиплікативний Гамма-шум
        k = 1 / (noise_level ** 2)
        noise = np.random.gamma(k, 1.0, shape) / k
        
        result = signal * noise
        clamped_result = np.clip(result, 1, 255)
        return clamped_result

class InteractiveApp:
    def __init__(self):
        # Початкові параметри зі статті
        self.DEFAULT_A = 8.39
        self.DEFAULT_B = 1.2
        self.DEFAULT_NOISE = 0.25
        self.DEFAULT_SIGMA = 1.0

        self.apply_filter = True
        
        self.cached_image = None
        self.last_noise_val = -1
        
        self._setup_widgets()
        self._setup_layout()
        
    def _setup_widgets(self):
        style = {'description_width': 'initial'}
        
        # Слайдери параметрів перетворення
        self.w_a = widgets.FloatSlider(value=self.DEFAULT_A, min=1.0, max=20.0, step=0.1, 
                                     description='Param a (Scale):', style=style, continuous_update=False)
        self.w_b = widgets.FloatSlider(value=self.DEFAULT_B, min=1.01, max=2.0, step=0.01, 
                                     description='Param b (Base):', style=style, continuous_update=False)
        
        # Слайдери симуляції
        self.w_noise = widgets.FloatSlider(value=self.DEFAULT_NOISE, min=0.01, max=1.0, step=0.01, 
                                         description='Noise Level (Speckle):', style=style, continuous_update=False)
        self.w_sigma = widgets.FloatSlider(value=self.DEFAULT_SIGMA, min=0.1, max=5.0, step=0.1, 
                                         description='Filter Sigma:', style=style, continuous_update=False)
        
        # Кнопка Reset
        self.btn_reset = widgets.Button(
            description='Reset to Paper Defaults',
            button_style='info', # 'success', 'info', 'warning', 'danger' or ''
            icon='undo'
        )
        self.btn_reset.on_click(self.reset_values)
        
        # Вивід графіку
        self.out_plot = widgets.Output()

    def _setup_layout(self):
        # Компонування віджетів
        params_box = widgets.VBox([
            widgets.HTML("<b>Параметри перетворення (VST):</b>"),
            self.w_a, self.w_b
        ])
        
        sim_box = widgets.VBox([
            widgets.HTML("<b>Параметри зображення та фільтра:</b>"),
            self.w_noise, self.w_sigma
        ])

        # Toggle filtering checkbox (self.apply_filter)
        filter_checkbox = widgets.Checkbox(
            value=True,
            description='Apply Inverse VST after Filtering',
            indent=False
        )
        def on_filter_toggle(change):
            self.apply_filter = change['new']
            self.update_view()
        filter_checkbox.observe(on_filter_toggle, names='value')
        
        controls = widgets.HBox([params_box, sim_box, filter_checkbox, self.btn_reset])
        controls.layout.align_items = 'center'
        controls.layout.justify_content = 'space-around'
        controls.layout.margin = '20px 0px 20px 0px'
        controls.layout.border = '1px solid #ddd'
        controls.layout.padding = '10px'
        
        # Головний контейнер
        self.layout = widgets.VBox([controls, self.out_plot])
        
        # Прив'язка подій зміни слайдерів до функції оновлення
        self.w_a.observe(self.update_view, names='value')
        self.w_b.observe(self.update_view, names='value')
        self.w_noise.observe(self.update_view, names='value')
        self.w_sigma.observe(self.update_view, names='value')

    def reset_values(self, b):
        """Повертає значення слайдерів до параметрів зі статті."""
        self.w_a.value = self.DEFAULT_A
        self.w_b.value = self.DEFAULT_B
        self.w_noise.value = self.DEFAULT_NOISE
        self.w_sigma.value = self.DEFAULT_SIGMA
        # update_view викличеться автоматично через observe

    def get_data(self, noise_level):
        """Генерує дані тільки якщо змінився рівень шуму."""
        if noise_level != self.last_noise_val:
            self.cached_image = ImageGenerator.generate(noise_level=noise_level)
            self.last_noise_val = noise_level
        return self.cached_image

    def update_view(self, change=None):
        """Основна логіка оновлення графіків."""
        
        # 1. Зчитування значень
        val_a = self.w_a.value
        val_b = self.w_b.value
        val_noise = self.w_noise.value
        val_sigma = self.w_sigma.value
        
        # 2. Отримання даних
        raw_image = self.get_data(val_noise)
        
        # 3. Обробка (Pipeline)
        config = VSTConfig(a=val_a, b=val_b)
        vst = VarianceStabilizer(config)
        
        # 3.1 Пряме перетворення
        img_log = vst.forward(raw_image)
        
        # 3.2 Розрахунок параметрів фільтра
        # Ми конвертуємо шум спеклу в еквівалент адитивного шуму
        target_std = vst.calc_additive_std(val_noise)
        
        # 3.3 Фільтрація (тут Гаус, можна замінити на BM3D)
        # Ми множимо val_sigma на target_std, щоб адаптувати силу фільтра
        # до реальної дисперсії шуму в лог-домені
        filter_radius = val_sigma * target_std 
        # Обмежуємо мінімальний радіус, щоб було видно ефект
        filter_radius = max(filter_radius, 0.5) 
        
        img_denoised_log = gaussian_filter(img_log, sigma=filter_radius)
        
        # 3.4 Зворотне перетворення, якщо галочка увімкнена
        if self.apply_filter:
            img_restored = vst.inverse(img_denoised_log)
        else:
            img_restored = vst.inverse(img_log)
        
        # 4. Візуалізація
        with self.out_plot:
            clear_output(wait=True) # Запобігає мерехтінню
            
            fig, axes = plt.subplots(1, 3, figsize=(15, 5), constrained_layout=True)
            
            # Вхідне
            ax0 = axes[0]
            im0 = ax0.imshow(raw_image, cmap='gray', vmin=0, vmax=np.percentile(raw_image, 99))
            ax0.set_title(f'1. Input (Speckle Noise {val_noise})')
            plt.colorbar(im0, ax=ax0, fraction=0.046, pad=0.04)
            
            # Лог-домен
            ax1 = axes[1]
            im1 = ax1.imshow(img_log, cmap='viridis')
            ax1.set_title(f'2. VST Domain (Log)\nFilter Sigma: {filter_radius:.2f}')
            plt.colorbar(im1, ax=ax1, fraction=0.046, pad=0.04)
            
            # Результат
            ax2 = axes[2]
            im2 = ax2.imshow(img_restored, cmap='gray', vmin=0, vmax=np.percentile(raw_image, 99))
            ax2.set_title('3. Restored Image')
            plt.colorbar(im2, ax=ax2, fraction=0.046, pad=0.04)
            
            plt.show()

    def show(self):
        """Показати інтерфейс."""
        display(self.layout)
        # Перший рендер
        self.update_view()



In [ ]:
app = InteractiveApp()
app.show()